In [40]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [41]:
# District Summary
# Total number of schools
total_schools = len(school_data_complete["school_name"].unique())

# Total number of students
total_students = len(school_data_complete)

# Total budget
total_budget = (school_data_complete["budget"].unique()).sum()

# Average math score
avg_math = school_data_complete["math_score"].mean()

# Average reading score
avg_read = school_data_complete["reading_score"].mean()

# Overall passing rate
overall_passing_rate = (avg_math + avg_read)/2

# Percentage of students passing math
pct_passing_math = len(school_data_complete[school_data_complete["math_score"] > 69]) / total_students * 100

# Percentage of students passing reading
pct_passing_reading = len(school_data_complete[school_data_complete["reading_score"] > 69]) / total_students * 100

# Create dataframe
summary_df = pd.DataFrame([{"Total Schools": total_schools, "Total Students": total_students, "Total Budget": total_budget, "Average Math Score": avg_math, "Average Reading Score": avg_read, "Percentage Passing Math": pct_passing_math, "Percentage Passing Reading": pct_passing_reading, "Overall Passing Rate": overall_passing_rate}])

summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


In [48]:
# School Summary
# Create dataframe
school_df = school_data[["school_name", "type", "size", "budget"]]

# Add per student budget column
school_df["Per Student Budget"] = school_df["budget"] / school_df["size"]

# Group data by school name
grouped_student_data = student_data.groupby(["school_name"], as_index= False)

# Create dataframe for average math score
avg_math_df = pd.DataFrame(grouped_student_data["math_score"].mean())

# Create dataframe for average reading score
avg_read_df = pd.DataFrame(grouped_student_data["reading_score"].mean())

# Create dataframe to calculate percentage passing math per school
pct_passing_math_df = student_data[student_data["math_score"] > 69]
grouped_pct_passing_math = pct_passing_math_df.groupby(["school_name"], as_index= False)
pct_passing_math_by_school_df = pd.DataFrame(grouped_pct_passing_math["math_score"].count())

# Create dataframe to calculate percentage passing reading per school
pct_passing_read_df = student_data[student_data["reading_score"] > 69]
grouped_pct_passing_read = pct_passing_read_df.groupby(["school_name"], as_index= False)
pct_passing_read_by_school_df = pd.DataFrame(grouped_pct_passing_read["reading_score"].count())

# Create dataframe to calculate overall percentage passing per school
overall_passing_rate_df = student_data.copy()
overall_passing_rate_df["overall_score"] = (overall_passing_rate_df["math_score"] + overall_passing_rate_df["reading_score"])/2
overall_pass_percentage = overall_passing_rate_df[overall_passing_rate_df["overall_score"]>69]
grouped_pct_passing_overall = overall_pass_percentage.groupby(["school_name"], as_index = False)
final_overall_passing_rate = pd.DataFrame(grouped_pct_passing_overall["overall_score"].count())

# Merge DataFrames
df1 = pd.merge(school_df, avg_math_df, how="outer", on=["school_name"])
df2 = pd.merge(df1, avg_read_df, how="outer", on=["school_name"])
df3 = pd.merge(df2, pct_passing_math_by_school_df, how="outer", on=["school_name"])
df4 = pd.merge(df3, pct_passing_read_by_school_df, how="outer", on=["school_name"])
final_df = pd.merge(df4, final_overall_passing_rate, how="outer", on=["school_name"])

# Calculate percentage passing math by school
final_df["Percentage Passing Math"] = final_df["math_score_y"] / final_df["size"] * 100

# Calculate percentage passing reading by school
final_df["Percentage Passing Reading"] = final_df["reading_score_y"] / final_df["size"] * 100

# Calculate overall percentage passing by school
final_df["Overall Passing Rate"] = final_df["overall_score"] / final_df["size"] * 100

# Rename columns
final_df = final_df.rename(columns = {"school_name": "School Name", "type" : "School Type", "size" : "Total Students", "budget" : "Total School Budget", "math_score_x" : "Average Math Score","reading_score_x" : "Average Reading Score"})

# Delete unnecessary columns
del final_df["math_score_y"]
del final_df["reading_score_y"]
del final_df["overall_score"]

In [49]:
# Top Performing Schools
# Sort to find top 5 schools
final_sorted_df = final_df.sort_values("Overall Passing Rate", ascending = False)
final_sorted_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,99.784715
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,99.772856
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,99.727520
10,Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,93.333333,96.611111,99.611111
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,99.605782


In [50]:
# Worst Performing Schools
# Sort to find worst 5 schools
final_sorted_worst_df = final_df.sort_values("Overall Passing Rate", ascending = True)
final_sorted_worst_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,85.961343
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,86.199343
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,86.246562
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,86.274116
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,86.287281
